In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.4 MB/s eta 0:00:00


In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199677 sha256=e452e45e7c895e23598cc0cb26485b3acd2e7c514573c78387be2578add41e68
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
import json
from tqdm import tqdm
import pandas as pd
tqdm.pandas()
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding,Bidirectional,Concatenate
from keras.optimizers import *

from tensorflow.keras.utils import to_categorical

import pickle
from datasets import load_metric

from nltk.translate.bleu_score import corpus_bleu
import fasttext
import tqdm as notebook_tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Read the training data from JSON file
train_data = pd.read_json("/content/drive/MyDrive/internship/asm_train.json", lines=True)
test_data=pd.read_json("/content/drive/MyDrive/internship/asm_test.json", lines=True)

In [6]:
#converts the JSON data into a Pandas DataFrame
def correct_json_format(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        json_data = file.read()

    # Process the JSON string to insert commas between objects
    json_data = json_data.replace('}\n{', '},{')
    json_list = json.loads(f'[{json_data}]')
    df = pd.json_normalize(json_list)

    return df


In [7]:
df_train = correct_json_format("/content/drive/MyDrive/internship/asm_train.json")
df_test = correct_json_format("/content/drive/MyDrive/internship/asm_test.json")


In [8]:
df_train


,unique_identifier,native word,english word,source,score
0,asm1,দীপলিনা,deeplina,Wikidata,NaN
1,asm2,বন্দ্যোপাধ্যায়,bandyopadhyay,Wikidata,NaN
2,asm3,কৃষন,krishan,Wikidata,NaN
3,asm4,বিষয়া,bishaya,Wikidata,NaN
4,asm5,পেমা,pema,Wikidata,NaN
...,...,...,...,...,...
178625,asm178626,বাগিচাসমূহো,bagisaxomuhu,IndicCorp,-0.058495
178626,asm178627,কার্যনির্বাহক,karzonirbahok,IndicCorp,-0.222479
178627,asm178628,এইটি,eiti,IndicCorp,-0.161426
178628,asm178629,চিত্রশালাত,sitroxalat,IndicCorp,-0.111472


In [9]:
df_test

,unique_identifier,native word,english word,source
0,asm1,লক্ষীনগৰস্থিত,lakhyeenogorsthito,AK-Freq
1,asm2,চতুৰ্থ,soturtho,AK-Freq
2,asm3,এইখন,eikhan,AK-Freq
3,asm4,প্ৰতিমূৰ্তিসমূহ,protimurtixomuh,AK-Freq
4,asm5,প্ৰতিযোগিতাতে,protijugitate,AK-Freq
...,...,...,...,...
5501,asm5502,ঝা,jha,AK-NEI
5502,asm5503,এলেকন,elecon,AK-NEI
5503,asm5504,ডেচাবাথুলা,desabathula,AK-NEI
5504,asm5505,কাৰখেলে,karkhele,AK-NEI


In [10]:
#subset train DataFrame
subset_df_train=df_train.sample(frac=0.3).reset_index(drop=True)
subset_df_train.shape

(53589, 5)

In [11]:
df_train=subset_df_train
df_train.shape

(53589, 5)

In [12]:
 #start symbol "\t" at the beginning of the text and an end symbol "\n" at the end of the text(target_text)
def add_start_end(target_text):
  text = "\t" + target_text + "\n"
  return text


In [13]:
#the DataFrame df_train will have a new column 'target_ass' containing the original text values from the 'native word' column with a start symbol "\t" at the beginning and an end symbol "\n" at the end of each text value.
df_train['target_ass'] = df_train['native word'].progress_apply(add_start_end)
df_train.head()

100%|██████████| 53589/53589 [00:00<00:00, 404989.85it/s]


,unique_identifier,native word,english word,source,score,target_ass
0,asm86051,মহলদাৰজনে,moholdarjone,IndicCorp,-0.287724,\tমহলদাৰজনে\n
1,asm143759,পাৰিপাৰ্শিক,pariparxik,IndicCorp,-0.103065,\tপাৰিপাৰ্শিক\n
2,asm84974,বিচিত্রধর্মী,bisitrodhormi,IndicCorp,-0.140332,\tবিচিত্রধর্মী\n
3,asm29944,পালিকাক,palikak,IndicCorp,-0.049551,\tপালিকাক\n
4,asm134787,অংশগ্ৰহণত,ongxogrohonot,IndicCorp,-0.071415,\tঅংশগ্ৰহণত\n


In [14]:
# add only the end token
# add_end takes a string target_text as input and adds an end symbol "\n"
def add_end(target_text):
  text = target_text + "\n"
  return text

In [15]:
 #adds a new column named 'decoder_target' to the DataFrame df_train. The values in this new column are derived from the values in the 'native word' column.
df_train['decoder_target'] = df_train['native word'].progress_apply(add_end)
df_train.head()


100%|██████████| 53589/53589 [00:00<00:00, 431399.36it/s]


,unique_identifier,native word,english word,source,score,target_ass,decoder_target
0,asm86051,মহলদাৰজনে,moholdarjone,IndicCorp,-0.287724,\tমহলদাৰজনে\n,মহলদাৰজনে\n
1,asm143759,পাৰিপাৰ্শিক,pariparxik,IndicCorp,-0.103065,\tপাৰিপাৰ্শিক\n,পাৰিপাৰ্শিক\n
2,asm84974,বিচিত্রধর্মী,bisitrodhormi,IndicCorp,-0.140332,\tবিচিত্রধর্মী\n,বিচিত্রধর্মী\n
3,asm29944,পালিকাক,palikak,IndicCorp,-0.049551,\tপালিকাক\n,পালিকাক\n
4,asm134787,অংশগ্ৰহণত,ongxogrohonot,IndicCorp,-0.071415,\tঅংশগ্ৰহণত\n,অংশগ্ৰহণত\n


In [16]:
def tokenize(lang):
# Create tokenizer
    #(filters)means that all characters in the input text will be considered for tokenization,(char_lvl)each character will be treated as a separate token.
    tokenizer = Tokenizer(filters='',char_level=True)
    # Fit texts
     #creates the vocabulary based on the characters present in the corpus.
    tokenizer.fit_on_texts(lang)
    word_index = tokenizer.word_index
    return tokenizer,word_index

In [17]:
# Tokenize words
roman_words = df_train['english word']
ass_words = df_train['target_ass']

roman_tokenizer, roman_tokens = tokenize(roman_words)
ass_tokenizer, ass_tokens = tokenize(ass_words)


print(roman_tokens)

print("No.of unique input tokens:", len(roman_tokens))


{'o': 1, 'a': 2, 'r': 3, 'i': 4, 't': 5, 'n': 6, 'u': 7, 'h': 8, 'k': 9, 'e': 10, 'b': 11, 'l': 12, 'm': 13, 'd': 14, 's': 15, 'p': 16, 'x': 17, 'g': 18, 'j': 19, 'y': 20, 'w': 21, 'c': 22, 'f': 23, 'v': 24, 'z': 25, 'q': 26}
No.of unique input tokens: 26


In [18]:
ass_words

0            \tমহলদাৰজনে\n
1          \tপাৰিপাৰ্শিক\n
2         \tবিচিত্রধর্মী\n
3              \tপালিকাক\n
4            \tঅংশগ্ৰহণত\n
               ...        
53584        \tঅভিনৱত্বই\n
53585    \tবৰ্গকিলোমিটাৰ\n
53586          \tইণ্ডিজত\n
53587         \tখেতিয়কৰ\n
53588          \tনাইহোৱা\n
Name: target_ass, Length: 53589, dtype: object

In [19]:
roman_words[15]

'kaxorjonor'

In [20]:
print(ass_tokens )
print("No.of unique output tokens:", len(ass_tokens))


{'\t': 1, '\n': 2, 'া': 3, 'ৰ': 4, '্': 5, 'ি': 6, 'ক': 7, 'ে': 8, 'ন': 9, 'ত': 10, 'ল': 11, 'ব': 12, 'ম': 13, 'প': 14, 'স': 15, 'ো': 16, 'ু': 17, 'দ': 18, 'ী': 19, 'ট': 20, 'হ': 21, 'জ': 22, 'গ': 23, 'ই': 24, 'চ': 25, 'য': 26, 'ৱ': 27, 'শ': 28, 'র': 29, 'ণ': 30, 'ভ': 31, 'খ': 32, 'ষ': 33, 'ধ': 34, 'ৈ': 35, 'ছ': 36, 'ং': 37, 'অ': 38, 'থ': 39, 'আ': 40, 'ড': 41, 'ূ': 42, 'ও': 43, 'ফ': 44, 'উ': 45, 'এ': 46, '়': 47, 'ৃ': 48, 'ঁ': 49, 'ঘ': 50, 'ঞ': 51, 'ঠ': 52, 'ঙ': 53, 'ৌ': 54, 'ৎ': 55, 'ঢ': 56, 'ঃ': 57, 'ঝ': 58, 'ঐ': 59, 'ঔ': 60, 'ঈ': 61, 'ঋ': 62, 'ঊ': 63}
No.of unique output tokens: 63


In [21]:
# vocab size (no. of unique characters for each script)

roman_vocab = len(roman_tokens)+1
ass_vocab = len(ass_tokens)+1
roman_vocab,ass_vocab

(27, 64)

In [22]:
roman_vocab

27

In [23]:
roman_words[45],ass_words[45]

('onibarzyo', '\tঅনিবাৰ্য্য\n')

In [24]:
# convert words to int sequence

roman_words_in_ids = roman_tokenizer.texts_to_sequences(roman_words)
ass_words_in_ids = ass_tokenizer.texts_to_sequences(ass_words)

In [25]:
roman_words_in_ids[45],ass_words_in_ids[45]

([1, 6, 4, 11, 2, 3, 25, 20, 1], [1, 38, 9, 6, 12, 3, 4, 5, 26, 5, 26, 2])

In [26]:
roman_words_in_ids

[[13, 1, 8, 1, 12, 14, 2, 3, 19, 1, 6, 10],
 [16, 2, 3, 4, 16, 2, 3, 17, 4, 9],
 [11, 4, 15, 4, 5, 3, 1, 14, 8, 1, 3, 13, 4],
 [16, 2, 12, 4, 9, 2, 9],
 [1, 6, 18, 17, 1, 18, 3, 1, 8, 1, 6, 1, 5],
 [13, 10, 5, 8, 4, 7, 19, 1, 9],
 [19, 4, 8, 1, 3],
 [12, 10, 6, 18, 2, 3],
 [18, 7, 3, 7, 5],
 [16, 1, 3, 2, 18, 1, 13, 1, 6, 4],
 [11, 10, 12, 4, 7],
 [2, 23, 3, 4, 6, 10],
 [17, 4, 17, 7, 5, 4, 5],
 [7, 9, 8, 1, 3, 7, 12, 1, 5],
 [19, 1, 6, 1, 11, 4, 15, 21, 2, 17, 1, 9],
 [9, 2, 17, 1, 3, 19, 1, 6, 1, 3],
 [6, 1, 18, 1, 3, 4, 3],
 [11, 2, 18, 4, 15, 2, 5, 8, 10],
 [13, 1, 19, 2, 5, 7],
 [1, 13, 2, 5, 20, 1, 11, 1, 3, 18, 1, 7],
 [18, 8, 1, 3, 9, 10, 4, 5, 2, 7],
 [8, 1, 3, 4, 3, 2, 13],
 [5, 4, 6, 4, 5, 1],
 [18, 10, 12, 2, 13, 2, 12, 1, 3],
 [8, 2, 16, 2, 6, 4],
 [9, 2, 16, 17, 1, 8, 1],
 [6, 4, 14, 3, 2, 11, 4, 8, 4, 6],
 [16, 10, 5, 10, 12, 1, 12, 1, 4],
 [18, 10, 10, 5, 1, 15, 8, 3, 4],
 [17, 1, 5, 4, 3, 5, 8, 1, 17, 1, 9, 1, 12],
 [17, 1, 14, 1, 15, 20, 1, 11, 8, 1, 3, 5, 4, 5, 1],
 

In [27]:
roman_words[5]

'methiujok'

In [28]:
ass_words_in_ids[5]


[1, 13, 8, 39, 6, 45, 22, 7, 2]

In [29]:
roman_words_in_ids[5]

[13, 10, 5, 8, 4, 7, 19, 1, 9]

In [30]:
# y_train
# only with end token

label_words = df_train["decoder_target"]
print(label_words[6])
#convert the text sequences in label_words into sequences of integers
y_train = ass_tokenizer.texts_to_sequences(label_words)
y_train

যিহৰ



[[13, 21, 11, 18, 3, 4, 22, 9, 8, 2],
 [14, 3, 4, 6, 14, 3, 4, 5, 28, 6, 7, 2],
 [12, 6, 25, 6, 10, 5, 29, 34, 29, 5, 13, 19, 2],
 [14, 3, 11, 6, 7, 3, 7, 2],
 [38, 37, 28, 23, 5, 4, 21, 30, 10, 2],
 [13, 8, 39, 6, 45, 22, 7, 2],
 [26, 6, 21, 4, 2],
 [11, 8, 53, 3, 4, 2],
 [23, 17, 4, 17, 10, 2],
 [14, 4, 3, 23, 13, 9, 6, 2],
 [12, 8, 11, 6, 43, 2],
 [40, 44, 5, 4, 6, 9, 8, 2],
 [28, 6, 28, 17, 20, 6, 10, 2],
 [45, 32, 4, 17, 11, 10, 2],
 [22, 9, 12, 6, 28, 5, 12, 3, 15, 7, 2],
 [7, 3, 33, 4, 22, 9, 4, 2],
 [9, 23, 4, 19, 4, 2],
 [12, 3, 23, 6, 25, 3, 10, 21, 8, 2],
 [13, 22, 3, 20, 16, 2],
 [38, 13, 3, 10, 5, 26, 12, 4, 5, 23, 43, 2],
 [50, 4, 7, 8, 24, 20, 3, 43, 2],
 [21, 4, 6, 4, 3, 13, 2],
 [10, 6, 9, 6, 10, 2],
 [23, 8, 11, 3, 13, 3, 11, 4, 2],
 [21, 3, 14, 3, 9, 19, 2],
 [7, 3, 14, 15, 21, 2],
 [9, 6, 18, 5, 29, 3, 12, 6, 21, 19, 9, 2],
 [14, 8, 20, 8, 11, 11, 35, 2],
 [23, 19, 10, 28, 5, 29, 19, 2],
 [15, 10, 19, 4, 5, 39, 15, 7, 11, 2],
 [15, 18, 15, 5, 26, 31, 29, 5, 10, 6, 1

In [31]:
y_train[9]

[14, 4, 3, 23, 13, 9, 6, 2]

In [32]:
print(label_words[9])

পৰাগমনি



In [33]:
# max sequence length

max_encoder_seq_length_roman = df_train['english word'].str.len().max()
max_decoder_seq_length_ass = df_train['target_ass'].str.len().max()

max_encoder_seq_length_roman, max_decoder_seq_length_ass

(23, 21)

In [34]:
def post_padding(data_in_int_seq, max_seq_len):
  return pad_sequences(data_in_int_seq, max_seq_len, padding = "post")
# padding

roman_padded = post_padding(roman_words_in_ids, max_encoder_seq_length_roman)
ass_padded = post_padding(ass_words_in_ids, max_decoder_seq_length_ass)
y_train_padded = post_padding(y_train, max_decoder_seq_length_ass)

print(roman_padded[16], len(roman_padded[16]))
print(ass_padded[16], len(ass_padded[16]))
print(y_train_padded[16],len(y_train_padded[16]))

[ 6  1 18  1  3  4  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 23
[ 1  9 23  4 19  4  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 21
[ 9 23  4 19  4  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] 21


In [35]:
latent_dim = 64
emb_dim = 100

In [36]:
# Define the encoder model
encoder_inputs = Input(shape=(max_encoder_seq_length_roman,), name="encoder_input")
emb_encoder = Embedding(roman_vocab, emb_dim, mask_zero=True, name="encoder_embedding")(encoder_inputs)

encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(latent_dim, return_state=True, return_sequences=False),name="BiLSTM",merge_mode="mul")(emb_encoder)


In [37]:
# concatenate along the last dimension
encoder_h = Concatenate(axis=-1,name="concat_h")([forward_h, backward_h])
encoder_c = Concatenate(axis=-1,name="concat_c")([forward_c, backward_c])
encoder_h, encoder_c


(<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_h')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_c')>)

In [38]:
encoder_states = [encoder_h,encoder_c]
encoder_states

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_h')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'concat_c')>]

In [39]:
# decoder with two lstm layers

decoder_inputs = keras.Input(shape=(None,),name="decoder_input")

decoder_emb = Embedding(ass_vocab, emb_dim, mask_zero=True, name="decoder_embedding")(decoder_inputs)
d_lstm_out,d_h,d_c = LSTM(128, return_state=True, return_sequences=True, name="decoder_lstm_1")(decoder_emb, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(ass_vocab, activation="softmax")
decoder_outputs = decoder_dense(d_lstm_out)

In [40]:
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 23)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 23, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                  

In [41]:
roman_padded[45]

array([ 1,  6,  4, 11,  2,  3, 25, 20,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [42]:
roman_words[45]

'onibarzyo'

In [43]:

ass_padded[45]

array([ 1, 38,  9,  6, 12,  3,  4,  5, 26,  5, 26,  2,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [44]:
y_train_padded[53587]

array([32,  8, 10,  6, 26, 47,  7,  4,  2,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [46]:
batch_size = 64
epochs = 50




model.fit(
    [roman_padded, ass_padded],
    y_train_padded,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("/content/drive/MyDrive/internship/mt_bilstm_1l.h5")

Epoch 1/50
670/670 [==============================] - 21s 32ms/step - loss: 0.3512 - accuracy: 0.8834 - val_loss: 0.3151 - val_accuracy: 0.8937
Epoch 2/50
670/670 [==============================] - 16s 24ms/step - loss: 0.2684 - accuracy: 0.9087 - val_loss: 0.2769 - val_accuracy: 0.9072
Epoch 3/50
670/670 [==============================] - 16s 25ms/step - loss: 0.2314 - accuracy: 0.9203 - val_loss: 0.2486 - val_accuracy: 0.9156
Epoch 4/50
670/670 [==============================] - 16s 23ms/step - loss: 0.2077 - accuracy: 0.9272 - val_loss: 0.2468 - val_accuracy: 0.9157
Epoch 5/50
670/670 [==============================] - 16s 23ms/step - loss: 0.1928 - accuracy: 0.9320 - val_loss: 0.2388 - val_accuracy: 0.9200
Epoch 6/50
670/670 [==============================] - 17s 25ms/step - loss: 0.1806 - accuracy: 0.9358 - val_loss: 0.2333 - val_accuracy: 0.9228
Epoch 7/50
670/670 [==============================] - 17s 25ms/step - loss: 0.1732 - accuracy: 0.9385 - val_loss: 0.2375 - val_accuracy:

In [47]:
#Load saved model
saved_model = tf.keras.models.load_model('/content/drive/MyDrive/internship/mt_bilstm_1l.h5')
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 23)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 23, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                  

In [48]:
inputs = saved_model.get_layer('encoder_input').output                    #Encoder input
bi_lstm_out,f_h,f_c,b_h,b_c = saved_model.get_layer('BiLSTM').output   #Encoder LSTM output

concate_h = saved_model.get_layer('concat_h').output
concate_c = saved_model.get_layer('concat_c').output


In [50]:
bi_lstm_out

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'BiLSTM')>

In [49]:
targets = saved_model.get_layer('decoder_input').output           # Decoder input
embedding_layer = saved_model.get_layer('decoder_embedding')      # Decoder Embedding layer
decoder_lstm1 = saved_model.get_layer('decoder_lstm_1')          # Decoder LSTM layer
dense1 = saved_model.get_layer('dense')                           # Updated layer name


In [82]:

#targets = saved_model.get_layer('decoder_input').output           #Decoder input
#embedding_layer = saved_model.get_layer('decoder_embedding')      #Decoder Embedding layer
#decoder_lstm1 = saved_model.get_layer('decoder_lstm_1')          #Decoder LSTM layer
#dense1 = saved_model.get_layer('dense_1')

In [51]:
#Encoder

encoder = keras.Model(inputs, [concate_h, concate_c])
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 23)]         0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, 23, 100)      2700        ['encoder_input[0][0]']          
                                                                                                  
 BiLSTM (Bidirectional)         [(None, 64),         84480       ['encoder_embedding[0][0]']      
                                 (None, 64),                                                      
                                 (None, 64),                                                      
                                 (None, 64),                                                

In [52]:
#Decoder

decoder_input_h1 = Input(shape=(128,))
decoder_input_c1 = Input(shape=(128,))

x = embedding_layer(targets)


In [53]:
x

<KerasTensor: shape=(None, None, 100) dtype=float32 (created by layer 'decoder_embedding')>

In [54]:
x,d_out_h,d_out_c = decoder_lstm1(x, initial_state=[decoder_input_h1, decoder_input_c1])

In [55]:
d_out_h

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'decoder_lstm_1')>

In [56]:
#x = decoder_lstm2(x, initial_state=[decoder_input_h1, decoder_input_c1])

x = dense1(x)
x.shape


TensorShape([None, None, 64])

In [57]:
decoder = keras.Model([targets, decoder_input_h1, decoder_input_c1], [x, d_out_h, d_out_c])
decoder.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 decoder_embedding (Embedding)  (None, None, 100)    6400        ['decoder_input[0][0]']          
                                                                                                  
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                            

In [58]:
reverse_input_char_index = dict((i, char) for char, i in roman_tokens.items())
reverse_target_char_index = dict((i, char) for char, i in ass_tokens.items())

In [59]:
reverse_input_char_index

{1: 'o',
 2: 'a',
 3: 'r',
 4: 'i',
 5: 't',
 6: 'n',
 7: 'u',
 8: 'h',
 9: 'k',
 10: 'e',
 11: 'b',
 12: 'l',
 13: 'm',
 14: 'd',
 15: 's',
 16: 'p',
 17: 'x',
 18: 'g',
 19: 'j',
 20: 'y',
 21: 'w',
 22: 'c',
 23: 'f',
 24: 'v',
 25: 'z',
 26: 'q'}

In [60]:
reverse_target_char_index

{1: '\t',
 2: '\n',
 3: 'া',
 4: 'ৰ',
 5: '্',
 6: 'ি',
 7: 'ক',
 8: 'ে',
 9: 'ন',
 10: 'ত',
 11: 'ল',
 12: 'ব',
 13: 'ম',
 14: 'প',
 15: 'স',
 16: 'ো',
 17: 'ু',
 18: 'দ',
 19: 'ী',
 20: 'ট',
 21: 'হ',
 22: 'জ',
 23: 'গ',
 24: 'ই',
 25: 'চ',
 26: 'য',
 27: 'ৱ',
 28: 'শ',
 29: 'র',
 30: 'ণ',
 31: 'ভ',
 32: 'খ',
 33: 'ষ',
 34: 'ধ',
 35: 'ৈ',
 36: 'ছ',
 37: 'ং',
 38: 'অ',
 39: 'থ',
 40: 'আ',
 41: 'ড',
 42: 'ূ',
 43: 'ও',
 44: 'ফ',
 45: 'উ',
 46: 'এ',
 47: '়',
 48: 'ৃ',
 49: 'ঁ',
 50: 'ঘ',
 51: 'ঞ',
 52: 'ঠ',
 53: 'ঙ',
 54: 'ৌ',
 55: 'ৎ',
 56: 'ঢ',
 57: 'ঃ',
 58: 'ঝ',
 59: 'ঐ',
 60: 'ঔ',
 61: 'ঈ',
 62: 'ঋ',
 63: 'ঊ'}

In [61]:
v = np.reshape(roman_padded[6], (1, -1))


In [62]:
v.shape

(1, 23)

In [63]:
encoder.predict(v,verbose=0)

[array([[ 9.16448142e-03, -1.28983078e-03, -1.04598880e-01,
         -2.04007640e-01,  5.12562871e-01, -2.22951084e-01,
          1.12081517e-03,  9.41444576e-01,  2.56580878e-02,
         -2.98934877e-01, -1.94440871e-01,  2.04694241e-01,
          1.10768555e-02, -3.24098319e-01,  3.34365061e-04,
         -1.50396109e-01, -8.74609649e-01,  7.44069650e-09,
          4.44210470e-01, -5.91729641e-01, -6.19393587e-03,
          6.74093366e-01,  5.37477918e-02, -2.04666182e-02,
          5.94219637e-06,  3.30371886e-01, -8.31933856e-01,
         -8.94785523e-02,  5.96566871e-03,  1.32541361e-04,
         -1.75376549e-01,  6.29054070e-01, -4.48106002e-04,
         -9.18290675e-01, -3.75315751e-04,  9.15001631e-01,
          4.09753740e-01, -9.01069999e-01, -1.10233733e-02,
         -9.37447667e-01,  3.84385377e-01, -6.87893629e-02,
         -2.91692857e-07,  4.63164225e-02,  7.07661569e-01,
         -1.20360889e-02,  6.92366157e-03,  6.56081643e-03,
          3.33612263e-02, -5.30240405e-0

In [64]:
def predict_list_of_words(list_source_words_in_padded_int_seq):
  list_pred_words = []
  start_token = np.zeros((1, 1))
  start_token[0] = ass_tokenizer.word_index['\t']
  print(start_token)

  for x in tqdm(range(len(list_source_words_in_padded_int_seq))):
    source_seq = start_token
    pred_word = ''
    v = np.reshape(list_source_words_in_padded_int_seq[x], (1, -1))
    next_h, next_c = encoder.predict(v,verbose=0)
    for i in range(max_decoder_seq_length_ass):
      output, next_h, next_c = decoder.predict([source_seq] + [next_h, next_c],verbose=0)
      next_token = np.argmax(output[0, 0, :])
      next_char = reverse_target_char_index[next_token]
      if next_char == '\n':
        break
      else:
        pred_word += next_char
        source_seq = np.zeros((1, 1))
        source_seq[0] = next_token
    list_pred_words.append(pred_word)
  return list_pred_words

In [65]:
predicted = predict_list_of_words(roman_padded[:20])

[[1.]]


100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


In [66]:
df_analyse = pd.DataFrame()
df_analyse["source"] = roman_words[:20]
df_analyse["predicted"] = predicted
df_analyse["ground_truth"] = df_train["native word"]

df_analyse

,source,predicted,ground_truth
0,moholdarjone,মহলদাৰজনে,মহলদাৰজনে
1,pariparxik,পাৰিপার্শিক,পাৰিপাৰ্শিক
2,bisitrodhormi,বিচিত্রধর্মী,বিচিত্রধর্মী
3,palikak,পালিকাক,পালিকাক
4,ongxogrohonot,অংশগ্ৰহণত,অংশগ্ৰহণত
5,methiujok,মেঠিউজক,মেথিউজক
6,jihor,যিহৰ,যিহৰ
7,lengar,লেঙাৰ,লেঙাৰ
8,gurut,গুৰুত,গুৰুত
9,poragomoni,পৰাগমণি,পৰাগমনি


In [67]:
test_roman_words = df_test['english word']
test_target_words = df_test['native word']
test_roman_words[2],test_target_words[2]

('eikhan', 'এইখন')

In [68]:
test_roman_words_in_ids = roman_tokenizer.texts_to_sequences(test_roman_words)
test_target_words_in_ids = ass_tokenizer.texts_to_sequences(test_target_words)

print(test_roman_words_in_ids[4])
print(test_target_words_in_ids[4])

[16, 3, 1, 5, 4, 19, 7, 18, 4, 5, 2, 5, 10]
[14, 5, 4, 10, 6, 26, 16, 23, 6, 10, 3, 10, 8]


In [69]:
#2. padding

test_roman_padded = post_padding(test_roman_words_in_ids, max_encoder_seq_length_roman)
test_target_padded = post_padding(test_target_words_in_ids, max_decoder_seq_length_ass)

print(test_roman_padded[2])
print(test_target_padded[2])

[10  4  9  8  2  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[46 24 32  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [70]:
len(test_roman_padded[2]), len(test_target_padded[2])

(23, 21)

In [71]:
# prediction on test set

predicted_test = predict_list_of_words(test_roman_padded[:1500])

[[1.]]


100%|██████████| 1500/1500 [16:21<00:00,  1.53it/s]


In [72]:
# analyse

df_analyse_test = pd.DataFrame()
df_analyse_test["source"] = test_roman_words[:1500]
df_analyse_test["predicted"] = predicted_test
df_analyse_test["ground_truth_label"] = df_test["native word"][:1500]

df_analyse_test.head(10)

,source,predicted,ground_truth_label
0,lakhyeenogorsthito,লক্ষীণগীষ্ঠৱৰ,লক্ষীনগৰস্থিত
1,soturtho,চতুৰ্থ,চতুৰ্থ
2,eikhan,এইখান,এইখন
3,protimurtixomuh,প্রতিমূৰ্তীসমূহ,প্ৰতিমূৰ্তিসমূহ
4,protijugitate,প্ৰতিযোগিতাতে,প্ৰতিযোগিতাতে
5,niya,নিয়া,নিয়া
6,aason,আচন,আঁচন
7,deutaloi,দেউতালৈ,দেউতালৈ
8,eaglenest,গেইলেন্স,ঈগলনেষ্ট
9,xeehotok,সীহঁতক,সিহঁতক


In [73]:
# Calculate accuracy

accuracy = (df_analyse_test['predicted'] == df_analyse_test['ground_truth_label']).mean()

print(f'Test Accuracy = {accuracy*100:.2f}%')

Test Accuracy = 36.07%


In [74]:

def split_characters(text_string):
  return [character for character in text_string]

split_characters('চ' 'তু' 'ৰ্থ')

['চ', 'ত', 'ু', 'ৰ', '্', 'থ']

In [75]:
df_analyse_test["predicted_tokenized"] = df_analyse_test["predicted"].apply(split_characters)
df_analyse_test["label_tokenized"] = df_analyse_test["ground_truth_label"].apply(split_characters)

df_analyse_test.head()

,source,predicted,ground_truth_label,predicted_tokenized,label_tokenized
0,lakhyeenogorsthito,লক্ষীণগীষ্ঠৱৰ,লক্ষীনগৰস্থিত,"[ল, ক, ্, ষ, ী, ণ, গ, ী, ষ, ্, ঠ, ৱ, ৰ]","[ল, ক, ্, ষ, ী, ন, গ, ৰ, স, ্, থ, ি, ত]"
1,soturtho,চতুৰ্থ,চতুৰ্থ,"[চ, ত, ু, ৰ, ্, থ]","[চ, ত, ু, ৰ, ্, থ]"
2,eikhan,এইখান,এইখন,"[এ, ই, খ, া, ন]","[এ, ই, খ, ন]"
3,protimurtixomuh,প্রতিমূৰ্তীসমূহ,প্ৰতিমূৰ্তিসমূহ,"[প, ্, র, ত, ি, ম, ূ, ৰ, ্, ত, ী, স, ম, ূ, হ]","[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]"
4,protijugitate,প্ৰতিযোগিতাতে,প্ৰতিযোগিতাতে,"[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]","[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]"


In [76]:
def create_reference(tokenized):
  return [tokenized]

df_analyse_test["reference"] = df_analyse_test["label_tokenized"].apply(create_reference)
df_analyse_test.head()

,source,predicted,ground_truth_label,predicted_tokenized,label_tokenized,reference
0,lakhyeenogorsthito,লক্ষীণগীষ্ঠৱৰ,লক্ষীনগৰস্থিত,"[ল, ক, ্, ষ, ী, ণ, গ, ী, ষ, ্, ঠ, ৱ, ৰ]","[ল, ক, ্, ষ, ী, ন, গ, ৰ, স, ্, থ, ি, ত]","[[ল, ক, ্, ষ, ী, ন, গ, ৰ, স, ্, থ, ি, ত]]"
1,soturtho,চতুৰ্থ,চতুৰ্থ,"[চ, ত, ু, ৰ, ্, থ]","[চ, ত, ু, ৰ, ্, থ]","[[চ, ত, ু, ৰ, ্, থ]]"
2,eikhan,এইখান,এইখন,"[এ, ই, খ, া, ন]","[এ, ই, খ, ন]","[[এ, ই, খ, ন]]"
3,protimurtixomuh,প্রতিমূৰ্তীসমূহ,প্ৰতিমূৰ্তিসমূহ,"[প, ্, র, ত, ি, ম, ূ, ৰ, ্, ত, ী, স, ম, ূ, হ]","[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]","[[প, ্, ৰ, ত, ি, ম, ূ, ৰ, ্, ত, ি, স, ম, ূ, হ]]"
4,protijugitate,প্ৰতিযোগিতাতে,প্ৰতিযোগিতাতে,"[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]","[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]","[[প, ্, ৰ, ত, ি, য, ো, গ, ি, ত, া, ত, ে]]"


In [77]:
# BLEU Score

# Convert DataFrame columns to lists
predictions = df_analyse_test['predicted_tokenized'].tolist()
true_labels = df_analyse_test['reference'].tolist()


In [78]:
true_labels

[[['ল', 'ক', '্', 'ষ', 'ী', 'ন', 'গ', 'ৰ', 'স', '্', 'থ', 'ি', 'ত']],
 [['চ', 'ত', 'ু', 'ৰ', '্', 'থ']],
 [['এ', 'ই', 'খ', 'ন']],
 [['প', '্', 'ৰ', 'ত', 'ি', 'ম', 'ূ', 'ৰ', '্', 'ত', 'ি', 'স', 'ম', 'ূ', 'হ']],
 [['প', '্', 'ৰ', 'ত', 'ি', 'য', 'ো', 'গ', 'ি', 'ত', 'া', 'ত', 'ে']],
 [['ন', 'ি', 'য', '়', 'া']],
 [['আ', 'ঁ', 'চ', 'ন']],
 [['দ', 'ে', 'উ', 'ত', 'া', 'ল', 'ৈ']],
 [['ঈ', 'গ', 'ল', 'ন', 'ে', 'ষ', '্', 'ট']],
 [['স', 'ি', 'হ', 'ঁ', 'ত', 'ক']],
 [['প', 'ূ', 'র', '্', 'ব', 'া', 'ঞ', '্', 'চ', 'ল', 'জ', 'ু', 'ৰ', 'ি']],
 [['প', 'ৰ', 'ি', 'দ', 'ৰ', '্', 'শ', 'ক']],
 [['হ', 'ে', 'ৰ', 'ু', 'ৱ', 'া', 'ই', 'ছ', 'ি', 'ল', 'ো']],
 [['স', 'দ', 'স', '্', 'য', 'স', 'ক', 'ল', 'ে']],
 [['স', 'ং', 'ক', '্', 'র', 'া', 'ন', '্', 'ত', 'ি', 'ৰ']],
 [['শ', 'ি', 'ক', '্', 'ষ', 'া', 'গ', 'ত', 'ভ', 'া', 'ৱ', 'ে']],
 [['ক', 'ৰ', 'ি', 'ল', 'ো']],
 [['ত', 'চ', 'ে', 'ন']],
 [['গ', 'ো', 'ৰ', '্', 'খ', 'া', 'স', 'ক', 'ল', 'ক']],
 [['ছ', 'া', 'ত', '্', 'ৰ', 'গ', 'ৰ', 'া', 'ক', 'ী', 'ক']],
 [['প', '্', 'ৰ', 'ত

In [79]:
predictions

[['ল', 'ক', '্', 'ষ', 'ী', 'ণ', 'গ', 'ী', 'ষ', '্', 'ঠ', 'ৱ', 'ৰ'],
 ['চ', 'ত', 'ু', 'ৰ', '্', 'থ'],
 ['এ', 'ই', 'খ', 'া', 'ন'],
 ['প', '্', 'র', 'ত', 'ি', 'ম', 'ূ', 'ৰ', '্', 'ত', 'ী', 'স', 'ম', 'ূ', 'হ'],
 ['প', '্', 'ৰ', 'ত', 'ি', 'য', 'ো', 'গ', 'ি', 'ত', 'া', 'ত', 'ে'],
 ['ন', 'ি', 'য', '়', 'া'],
 ['আ', 'চ', 'ন'],
 ['দ', 'ে', 'উ', 'ত', 'া', 'ল', 'ৈ'],
 ['গ', 'ে', 'ই', 'ল', 'ে', 'ন', '্', 'স'],
 ['স', 'ী', 'হ', 'ঁ', 'ত', 'ক'],
 ['প', 'ব', '্', 'ৰ', 'া', 'চ', 'ন', '্', 'য', 'ল', 'ো', 'ৰ', 'ি'],
 ['প', 'ৰ', 'ি', 'দ', 'র', '্', 'শ', 'ক'],
 ['হ', 'ে', 'ৰ', 'ু', 'ৱ', 'া', 'ই', 'ছ', 'ি', 'ল', 'ো', 'ঁ'],
 ['চ', 'দ', 'স', '্', 'য', 'া', 'স', 'ক', 'ল'],
 ['স', 'ং', 'ক', '্', 'ৰ', 'া', 'ন', '্', 'ত', 'ি', 'ৰ'],
 ['শ', 'ি', 'ক', '্', 'ষ', 'া', 'গ', 'ত', 'ভ', 'া', 'ৱ', 'ে'],
 ['ক', 'ৰ', 'ি', 'ল', 'ো', 'ঁ'],
 ['ত', 'ছ', 'ে', 'ন'],
 ['গ', 'ো', 'ৰ', 'খ', 'া', 'ল', 'খ', 'ৰ', 'ক'],
 ['ছ', 'া', 'ত', '্', 'র', 'গ', 'ৰ', 'া', 'ক', 'ী', 'ক'],
 ['প',
  '্',
  'ৰ',
  'ত',
  'ি',
  'জ',
  'ো',
  'গ',
  'ি

In [80]:
bleu = load_metric("bleu")
bleu.compute(predictions = predictions, references = true_labels)


<ipython-input-80-29ca3862bcb2>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("bleu")


{'bleu': 0.62843290890368,
 'precisions': [0.8485906709902719,
  0.6732212406193597,
  0.5637531848897751,
  0.4842733908427339],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0116073681554378,
 'translation_length': 12027,
 'reference_length': 11889}

In [83]:
def transliterate(source_word_in_padded_int_seq):
  #list_pred_words = []
  start_token = np.zeros((1, 1))
  start_token[0] = ass_tokenizer.word_index['\t']
  #print(start_token)

  #for x in tqdm(range(len(list_source_words_in_padded_int_seq))):
  source_seq = start_token
  pred_word = ''
  #v = np.reshape(source_word_in_padded_int_seq, (1, -1))
  next_h, next_c = encoder.predict(source_word_in_padded_int_seq,verbose=0)
  for i in range(max_decoder_seq_length_ass):
    output, next_h, next_c = decoder.predict([source_seq] + [next_h, next_c],verbose=0)
    next_token = np.argmax(output[0, 0, :])
    next_char = reverse_target_char_index[next_token]
    if next_char == '\n':
      break
    else:
      pred_word += next_char
      source_seq = np.zeros((1, 1))
      source_seq[0] = next_token
  #list_pred_words.append(pred_word)
  return pred_word

input_word = input("\nEnter a romanized word: ")
input_word_in_ids = roman_tokenizer.texts_to_sequences([input_word])

input_word_in_ids_padded = post_padding(input_word_in_ids, max_encoder_seq_length_roman)

print(f"\nTransliterated word : {transliterate(input_word_in_ids_padded)}")



Enter a romanized word: tumi

Transliterated word : তুমি
